<a href="https://colab.research.google.com/github/MaxLikhachev/MelodyRecognition/blob/main/MIDIToMRNF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U mido==1.2.9
!pip install -U jsonpickle

Requirement already up-to-date: mido==1.2.9 in /usr/local/lib/python3.7/dist-packages (1.2.9)
Requirement already up-to-date: jsonpickle in /usr/local/lib/python3.7/dist-packages (2.0.0)


In [2]:
import mido, math, json, jsonpickle

In [3]:
class Note:
  def __init__(self, note = 0, time = 0.0, ticks_per_beat = 1):
    self.note, self.time = note, self.__init_time(ticks_per_beat, time)

  def __init_time(self, ticks_per_beat = 1, time = 0.0):
    return time / ticks_per_beat

  def get_tone(self):
    return self.note % 12

  def get_octave(self):
    return self.note // 12

  def get_note_title(self):
    return ('C','C#','D','D#','E','F','F#','G','G#','A','A#','B')[self.get_tone()]

  def get_notation(self):
    return self.get_note_title(), self.get_octave(), self.get_duration()

  def get_duration(self):
    duration_notation, durations, temp = [], {32:'x8 whole', 16:'x4 whole', 8:'double whole', 4:'whole', 2:'half', 1:'quarter', 0.5:'8th', 0.25:'16th', 0.125:'32th', 0.0625:'64th'}, self.time
    for key in durations.keys():
      if temp // key == 1:
        duration_notation.append(durations.get(key))
        temp -= key
    return ' + '.join(duration_notation)

  def __str__(self):
    return 'note:' + self.note.__str__() + ' time:' + self.time.__str__() + ' notation:' + self.get_notation().__str__()


print(Note(note=50,time=1))

note:50 time:1.0 notation:('D', 4, 'quarter')


In [4]:
class NoteMRNF(Note):
  def __init__(self, note = Note(), first_note = Note()):
    self.note, self.time = self.__init_mrnf_note(note, first_note), self.__init_mrnf_time(note, first_note)

  def __init_mrnf_note(self, note, first_note):
    return note.note - first_note.note

  def __init_mrnf_time(self, note, first_note):
    return note.time / first_note.time

In [5]:
class Melody(object):
  def __init__(self, tempo = 0, title = '', author = '', notes = []):
    self.tempo, self.title, self.author, self.notes, = tempo, title, author, notes,

  def __str__(self):
    notes_str = '\n'.join(note.__str__() for note in self.notes)
    return 'tempo:' + self.tempo.__str__() + '(' + str(int(mido.tempo2bpm(self.tempo))) + ' bpm) title:' + self.title + ' author:' +  self.author + '\n' + notes_str


print(Melody(500000, 'Title', 'Author', []))

tempo:500000(120 bpm) title:Title author:Author



In [6]:
class MidiFile:
  def __init__(self, filename = ''):
    self.filename, self.original_melody, self.mrnf_melody = filename, Melody(), Melody()
    self.original_melody = self.__mid2melody()
    self.mrnf_melody = self.__melody2mnrf()
  
  def get_mid(self, filename):
    mid, msgs = mido.MidiFile(filename, clip=True), []
    for track in mid.tracks:
      for msg in track:
        msgs.append(msg)
    return msgs

  def __mid2melody(self):
    mid, melody = mido.MidiFile(self.filename, clip=True), Melody()
    for track in mid.tracks:
      for msg in track:
        if msg.type == 'set_tempo':
          melody.tempo = msg.tempo
        elif msg.type == 'track_name':
          if melody.title == '':            
            melody.title = msg.name
        elif msg.type == 'copyright':
          melody.author = msg.text
        elif msg.type == 'note_off':
          note, time = msg.note, msg.time
          if not time == 0:
            melody.notes.append(Note(note=note, time=time, ticks_per_beat=mid.ticks_per_beat))
    return melody

  def __melody2mnrf(self):
    return Melody(tempo = self.original_melody.tempo, title = self.original_melody.title, author = self.original_melody.author, notes = [NoteMRNF(note, self.original_melody.notes[0]) for note in self.original_melody.notes]) 
  

mid = MidiFile(filename='/content/sample.mid')



In [7]:
print('ORIGINAL_MELODY:') 
print(mid.original_melody) 

ORIGINAL_MELODY:
tempo:571428(105 bpm) title:Distortion Guitar author:Woulfan
note:40 time:0.5 notation:('E', 3, '8th')
note:47 time:0.5 notation:('B', 3, '8th')
note:54 time:0.5 notation:('F#', 4, '8th')
note:55 time:2.5 notation:('G', 4, 'half + 8th')
note:47 time:0.5 notation:('B', 3, '8th')
note:52 time:0.5 notation:('E', 4, '8th')
note:57 time:0.5 notation:('A', 4, '8th')
note:55 time:2.5 notation:('G', 4, 'half + 8th')
note:43 time:0.5 notation:('G', 3, '8th')
note:50 time:0.5 notation:('D', 4, '8th')
note:59 time:0.5 notation:('B', 4, '8th')
note:57 time:1.5 notation:('A', 4, 'quarter + 8th')
note:54 time:0.5 notation:('F#', 4, '8th')
note:52 time:0.5 notation:('E', 4, '8th')
note:54 time:4.0 notation:('F#', 4, 'whole')
note:40 time:0.5 notation:('E', 3, '8th')
note:47 time:0.5 notation:('B', 3, '8th')
note:54 time:0.5 notation:('F#', 4, '8th')
note:55 time:2.5 notation:('G', 4, 'half + 8th')
note:47 time:0.5 notation:('B', 3, '8th')
note:52 time:0.5 notation:('E', 4, '8th')
not

In [8]:
print('MNRF_MELODY:') 
print(mid.mrnf_melody) 

MNRF_MELODY:
tempo:571428(105 bpm) title:Distortion Guitar author:Woulfan
note:0 time:1.0 notation:('C', 0, 'quarter')
note:7 time:1.0 notation:('G', 0, 'quarter')
note:14 time:1.0 notation:('D', 1, 'quarter')
note:15 time:5.0 notation:('D#', 1, 'whole + quarter')
note:7 time:1.0 notation:('G', 0, 'quarter')
note:12 time:1.0 notation:('C', 1, 'quarter')
note:17 time:1.0 notation:('F', 1, 'quarter')
note:15 time:5.0 notation:('D#', 1, 'whole + quarter')
note:3 time:1.0 notation:('D#', 0, 'quarter')
note:10 time:1.0 notation:('A#', 0, 'quarter')
note:19 time:1.0 notation:('G', 1, 'quarter')
note:17 time:3.0 notation:('F', 1, 'half + quarter')
note:14 time:1.0 notation:('D', 1, 'quarter')
note:12 time:1.0 notation:('C', 1, 'quarter')
note:14 time:8.0 notation:('D', 1, 'double whole')
note:0 time:1.0 notation:('C', 0, 'quarter')
note:7 time:1.0 notation:('G', 0, 'quarter')
note:14 time:1.0 notation:('D', 1, 'quarter')
note:15 time:5.0 notation:('D#', 1, 'whole + quarter')
note:7 time:1.0 n

In [9]:
with open('data.json', 'w') as outfile:
    json.dump(jsonpickle.encode(mid.mrnf_melody), outfile)

print(jsonpickle.encode(mid.mrnf_melody))

{"py/object": "__main__.Melody", "tempo": 571428, "title": "Distortion Guitar", "author": "Woulfan", "notes": [{"py/object": "__main__.NoteMRNF", "note": 0, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 7, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 14, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 15, "time": 5.0}, {"py/object": "__main__.NoteMRNF", "note": 7, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 12, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 17, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 15, "time": 5.0}, {"py/object": "__main__.NoteMRNF", "note": 3, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 10, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 19, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 17, "time": 3.0}, {"py/object": "__main__.NoteMRNF", "note": 14, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "note": 12, "time": 1.0}, {"py/object": "__main__.NoteMRNF", "not

In [10]:
with open('data.json') as json_file:
    data = json.load(json_file)

print(jsonpickle.decode(data))

tempo:571428(105 bpm) title:Distortion Guitar author:Woulfan
note:0 time:1.0 notation:('C', 0, 'quarter')
note:7 time:1.0 notation:('G', 0, 'quarter')
note:14 time:1.0 notation:('D', 1, 'quarter')
note:15 time:5.0 notation:('D#', 1, 'whole + quarter')
note:7 time:1.0 notation:('G', 0, 'quarter')
note:12 time:1.0 notation:('C', 1, 'quarter')
note:17 time:1.0 notation:('F', 1, 'quarter')
note:15 time:5.0 notation:('D#', 1, 'whole + quarter')
note:3 time:1.0 notation:('D#', 0, 'quarter')
note:10 time:1.0 notation:('A#', 0, 'quarter')
note:19 time:1.0 notation:('G', 1, 'quarter')
note:17 time:3.0 notation:('F', 1, 'half + quarter')
note:14 time:1.0 notation:('D', 1, 'quarter')
note:12 time:1.0 notation:('C', 1, 'quarter')
note:14 time:8.0 notation:('D', 1, 'double whole')
note:0 time:1.0 notation:('C', 0, 'quarter')
note:7 time:1.0 notation:('G', 0, 'quarter')
note:14 time:1.0 notation:('D', 1, 'quarter')
note:15 time:5.0 notation:('D#', 1, 'whole + quarter')
note:7 time:1.0 notation:('G',